In [9]:
import requests
import os
import json
import random 
import time
from datetime import datetime
import psycopg2

In [ ]:
#Instruction to start 

# Headers


In [10]:
accept_languages = [
    'en-GB,en-US;q=0.9,en;q=0.8',
    "en-US,en;q=0.9",
    "en-GB,en;q=0.9",
    "fr-FR,fr;q=0.9",
    "de-DE,de;q=0.9",
    "es-ES,es;q=0.9",
    "it-IT,it;q=0.9",
    "ja-JP,ja;q=0.9",
    "zh-CN,zh;q=0.9"
    ]

accept = [
    "application/json, text/plain, */*",
    "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "application/json",
    "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "application/xml,application/json,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5",
    ]

def generate_user_agent():
    # Randomly choose the platform
    platform = random.choice(['macOS', 'Windows'])

    if platform == 'macOS':
        # macOS versions
        mac_os_versions = [f'10_{i}_{j}' for i in range(14, 16) for j in range(0, 10)]
        chrome_versions = [f'{i}.0.0.0' for i in range(118, 123)]
        opera_versions = [f'{i}.0.0.0' for i in range(104, 109)]
        safari_versions = ['605.1.15', '606.1.14', '607.1.40']
        
        browser_choice = random.choice(['Opera', 'Safari'])
        
        mac_os_version = random.choice(mac_os_versions)
        if browser_choice == 'Opera':
            chrome_version = random.choice(chrome_versions)
            opera_version = random.choice(opera_versions)
            user_agent = f'Mozilla/5.0 (Macintosh; Intel Mac OS X {mac_os_version}) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{chrome_version} Safari/537.36 OPR/{opera_version}'
        else: # Safari
            safari_version = random.choice(safari_versions)
            user_agent = f'Mozilla/5.0 (Macintosh; Intel Mac OS X {mac_os_version}) AppleWebKit/{safari_version} (KHTML, like Gecko) Version/{random.choice(["17.0", "17.1", "17.2"])} Safari/{safari_version}'
        
    elif platform == 'Windows':
        # Windows versions
        windows_versions = ['10.0', '6.1', '6.2', '6.3']
        chrome_versions = [f'{i}.0.0.0' for i in range(121, 131)]
        
        windows_version = random.choice(windows_versions)
        chrome_version = random.choice(chrome_versions)
        
        user_agent = f'Mozilla/5.0 (Windows NT {windows_version}; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{chrome_version} Safari/537.36'
    
    return user_agent

def generate_accept_encoding():
    encodings = ['gzip', 'deflate', 'br', 'identity']
    variations = [
        'gzip, deflate, br',
        'gzip, deflate',
        'br',
        'gzip',
        'deflate, br',
        'deflate',
        'identity',
        '*',  # Indicates that any encoding is acceptable.
        ]
    
    # You can also create combinations dynamically
    for _ in range(5):  # Generate 5 random combinations
        random.shuffle(encodings)
        variations.append(', '.join(encodings[:random.randint(1, len(encodings))]))

    return random.choice(variations)


base_header = {
    'host': 'apip.colruyt.be',
    'origin': "https://www.colruyt.be",
    'referer': "https://www.colruyt.be/",
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    "X-Cg-Apikey":"a8ylmv13-b285-4788-9e14-0f79b7ed2411",
    }

def get_header(): ## make this a proper import 
    updated_header = base_header.copy()
    updated_header['user-agent'] = generate_user_agent().strip()
    updated_header['Accept-Language'] = random.choice(accept_languages).strip()
    updated_header['Accept'] = random.choice(accept).strip()
    updated_header['accept-encoding'] = generate_accept_encoding().strip()
    
    return updated_header

In [11]:
shops = {
    459: "AUDERGHEM (COLRUYT)",
    1825: "SINT-DENIJS-WESTREM (COLRUYT)", # wealthiest commune of flanders
    471: "ETTERBEEK (RUE GRAY) COLRUYT)",
    1671: "DINANT (COLRUYT)" # one of poorest commune of wallonia
    }

In [12]:
# Database connection parameters
dbname = "colruyt_tracker"
user = "thibodebras"
password = "postgres"
host = "localhost"  # or the IP address of your PostgreSQL server

In [13]:
run_date = datetime.now().date()

In [21]:
# Establish a database connection
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
cur = conn.cursor()

table_date = str(run_date).replace("-", "_")
table_creation_querry = f"""
CREATE TABLE IF NOT EXISTS raw_data_{table_date}_todelete (
    run_date DATE,
    shop_id VARCHAR(255),
    page INT,
    products JSON
);
"""

cur.execute(table_creation_querry)
conn.commit()

In [22]:
run_date = datetime.now().date()
failed_retries = {}
error_dic = {}

def fetch_product_data_from_api(shop_id, page):
    """Attempt to fetch product data with retries for specific status codes."""
    max_retries = 3
    retry_delay = 30  # seconds
    api_url = f"https://apip.colruyt.be/gateway/ictmgmt.emarkecom.cgproductretrsvc.v2/v2/v2/fr/products?clientCode=CLP&page={page}&placeId={shop_id}&size=250&sort=popularity%20asc"
    last_status_code = None  # Keep track of the last status code

    for attempt in range(1, max_retries + 1):
        response = requests.get(api_url, headers=get_header())
        last_status_code = response.status_code
        print(f"{shop_id} {page} {response.status_code}")
        
        if response.status_code == 200:
            return response.json(), 200  # Return response data and status code for successful request.
        elif response.status_code == 456 and attempt < max_retries:
            print(f"Received status code 456 for shop {shop_id} on page {page}. Retrying after {retry_delay} seconds.")
            time.sleep(retry_delay)
        else:
            break  # Stop retrying for other errors or after max retries for 456.
    
    # Return None to indicate an unsuccessful fetch and the last status code.
    return None, last_status_code

def insert_data_to_psql(shop_id, page, products, run_date):
    try:
        cur.execute(f"INSERT INTO raw_data_{table_date}_todelete (shop_id, page, products, run_date) VALUES (%s, %s, %s, %s)",
                    (shop_id, page, json.dumps(products), run_date))
        conn.commit()  # Commit the transaction
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        conn.rollback()  # Rollback the transaction on error


In [23]:

total_insert_time = 0
for shop_id, name in shops.items():
    for page in range(1, 60):
        data, status_code = fetch_product_data_from_api(shop_id, page)  # Unpack the returned tuple

        if data:
            products = data.get("products", [])
            if not products:
                print(f"No products returned for shop {shop_id} on page {page}. Moving to next shop.")
                break  # No products found, move to the next shop.
            
            insert_data_to_psql(shop_id, page, products, run_date)
            
        else:
            # Log failed retries or errors based on the status code.
            if status_code == 456:
                failed_retries.setdefault(shop_id, []).append(page)
                print(f"Failed to fetch data for shop {shop_id} on page {page} after retries.")
            else:
                error_dic.setdefault(shop_id, []).append(page)
        
        time.sleep(random.uniform(0.5, 1.5))  # Cooldown between requests or pages.


459 1 200
459 2 200
459 3 200
459 4 200
459 5 200
459 6 200
459 7 200
459 8 200
459 9 200
459 10 200
459 11 200
459 12 200
459 13 200
459 14 200
459 15 200
459 16 200
459 17 200
459 18 200
459 19 200
459 20 200
459 21 200
459 22 200
459 23 200
459 24 200
459 25 200
459 26 200
459 27 200
459 28 200
459 29 200
459 30 200
459 31 200
459 32 200
459 33 200
459 34 200
459 35 200
459 36 200
459 37 200
459 38 200
459 39 200
459 40 200
459 41 200
459 42 200
459 43 200
459 44 200
459 45 200
459 46 200
459 47 200
459 48 200
459 49 200
459 50 200
459 51 200
459 52 200
459 53 200
459 54 200
459 55 200
459 56 200
No products returned for shop 459 on page 56. Moving to next shop.
1825 1 200
1825 2 200
1825 3 200
1825 4 200
1825 5 200
1825 6 200
1825 7 200
1825 8 200
1825 9 200
1825 10 200
1825 11 200
1825 12 200
1825 13 200
1825 14 200
1825 15 200
1825 16 200
1825 17 200
1825 18 200
1825 19 200
1825 20 200
1825 21 200
1825 22 200
1825 23 200
1825 24 200
1825 25 200
1825 26 200
1825 27 200
1825 28 200

In [24]:
print(f"Total time spent on insert_data_to_psql: {total_insert_time} seconds")

Total time spent on insert_data_to_psql: 200.03849625587463 seconds


In [16]:
# Commit the changes and close the connection
conn.commit()
cur.close()
conn.close()      